In [1]:
from src.dataset import MethylIterableDataset
import polars as pl
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import pyarrow.parquet as pq

In [2]:
train_parquet = 'data/processed/standard_600k_32_train.parquet'
test_parquet = 'data/processed/standard_600k_32_test.parquet'

In [3]:
subset_q = (
    pl.scan_parquet(train_parquet,
                    schema = {'read_name': pl.String,
                              'cg_pos': pl.Int64,
                              'seq': pl.String,
                              'fi': pl.List(pl.UInt16),
                              'fp': pl.List(pl.UInt16),
                              'ri': pl.List(pl.UInt16),
                              'rp': pl.List(pl.UInt16),
                              'label': pl.Int32
                              })
    .head(1_000_000)
    )


subset_df = subset_q.collect()

KINETICS_FEATURES = ['fi', 'fp', 'ri', 'rp']
def compute_log_normalization_stats(df, features, epsilon=1):
    means = {col: (df[col].explode() + epsilon).log().mean() for col in features}
    stds = {col: (df[col].explode() + epsilon).log().explode().std() for col in features}
    return means, stds

train_means, train_stds = compute_log_normalization_stats(subset_df, KINETICS_FEATURES)

In [4]:
train_ds = MethylIterableDataset(train_parquet,
                                    means=train_means,
                                    stds=train_stds,
                                    context = 32,
                                    restrict_row_groups = 1)

Failed to read given parquet file.


In [5]:
len(train_ds)

0